In [1]:
#!pip install vaex
#!pip install nbformat
#!pip install nbconvert
# Remove comments if vaex is not installed

In [2]:
import numpy as np # type: ignore
import os
import vaex # type: ignore
import pandas as pd
from datetime import datetime, timedelta
from pathlib import Path
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor

In [3]:
def run_notebook(notebook_path):
    # Load the notebook
    with open(notebook_path) as f:
        notebook = nbformat.read(f, as_version=4)
    
    # Execute the notebook
    ep = ExecutePreprocessor(timeout=600, kernel_name='python3')
    ep.preprocess(notebook, {'metadata': {'path': './'}})

# Run the other notebook
run_notebook("data_extraction_to_csv.ipynb")

In [4]:
# Get today's date
today = datetime.today()

# Check if today is Sunday (weekday() returns 6 for Sunday)
if today.weekday() != 6:
    # Subtract the necessary number of days to get to the previous Sunday
    today = today - timedelta(days=(today.weekday() + 1))

# Format the date as 'ddmmyyyy'
today = today.strftime('%d%m%Y')
print(today)

12012025


In [5]:
# set the route directory
csv_p = Path()
csv_folder_path = csv_p / f"Windsor Transit Data Weekly/{today}_transit_windsor"

# read multiple csv's
csv_data = {}
for file in csv_folder_path.iterdir():
    print(file.name)
    csv_name = file.name
    full_csv_path = file.resolve()
    csv_data[csv_name[:-4]] = pd.read_csv(full_csv_path)

# data access
stop_time_data = csv_data['stop_times']
stop_data = csv_data['stops']
routes_data = csv_data['routes']
trips_data = csv_data['trips']

stops.csv
routes.csv
calendar.csv
fare_attributes.csv
fare_rules.csv
agency.csv
shapes.csv
trips.csv
calendar_dates.csv
stop_times.csv
feed_info.csv


In [6]:
def extract_route_data():
    # Step 2.
    expected = routes_data[['route_id', 'route_long_name']]

    # Step 3.
    # Perform the left join
    expected = expected.merge(trips_data, on='route_id', how='left')[['direction_id', 'trip_id','route_long_name', 'route_id']]

    # Step 4.
    expected = expected.merge(stop_time_data, on='trip_id', how='left')[['route_long_name', 'direction_id', 'arrival_time', 
                                                                         'stop_id', 'stop_sequence', 'trip_id', 'departure_time', 'route_id']]

    # Step 5.
    expected = expected.merge(stop_data, on='stop_id', how='left')[['route_id','route_long_name', 'stop_name', 'stop_code','stop_lat', 
                                                                    'stop_lon', 'arrival_time', 'departure_time', 
                                                                    'stop_sequence','trip_id', 'direction_id']]
    expected = expected.sort_values(by=['route_id','trip_id','stop_sequence'])
    expected = expected.drop(columns='route_id')
    return expected

expected = extract_route_data()
expected.head()

,route_long_name,stop_name,stop_code,stop_lat,stop_lon,arrival_time,departure_time,stop_sequence,trip_id,direction_id
0,TRANSWAY 1A,Windsor Transit Terminal,1000,42.317486,-83.043368,17:50:00,17:50:00,1,1133706,1
1,TRANSWAY 1A,Ouellette at Chatham,1002,42.317882,-83.039374,17:52:00,17:52:00,2,1133706,1
2,TRANSWAY 1A,Ouellette at Park,1004,42.315935,-83.038091,17:53:00,17:53:00,3,1133706,1
3,TRANSWAY 1A,Ouellette at Wyandotte,1006,42.313499,-83.036628,17:54:00,17:54:00,4,1133706,1
4,TRANSWAY 1A,Ouellette at Elliott,1008,42.311320,-83.035171,17:59:00,17:59:00,5,1133706,1


In [7]:
expected['route_long_name'].unique() 

array(['TRANSWAY 1A', 'TRANSWAY 1C', 'CROSSTOWN 2', 'CENTRAL 3',
       'OTTAWA 4', 'DOUGALL 6', 'WALKERVILLE 8', 'PARENT 14', 'TUNNEL',
       'SOUTH WINDSOR 7', 'LAUZON 10', 'LASALLE 25', 'LTW ROUTE 42',
       '518 EXPRESS', 'ROUTE 605', '418 EXPRESS', 'ROUTE 115',
       'ROUTE 305'], dtype=object)

In [8]:
# Folder name containing the CSV files
folder_name = "12 Months APC Data - Transit Windsor"

# Initialize an empty dictionary to store DataFrames
dfs_dict = {}

# Loop through all files in the folder
for file_name in os.listdir(folder_name):
    if file_name.endswith(".csv"):
        # Extract the month and year from the file name
        file_parts = file_name.split("-")  # Split by "-"
        month_year = file_parts[-1].strip().replace(".csv", "")  # Get "Month Year" part

        # Load the CSV file into a Vaex DataFrame
        file_path = os.path.join(folder_name, file_name)
        df = vaex.open(file_path)

        # Store the DataFrame in the dictionary with the key as "month_year"
        dfs_dict[month_year] = df

# Print keys of the dictionary to verify
print("Loaded DataFrames for:", *list(dfs_dict.keys()), sep='\n')


Loaded DataFrames for:
October 2023
Jan 2024
October 2024
July 2024
June 2024
September 2024
August 2024
December 2023
Feb 2024
November 2023
March 2024
April 2024
May 2024


In [9]:
dfs_dict['September 2024'].head(5)

#,Stop Code,Stop Name,Day of Week,Direction,Month,Passengers In (bin),Passengers Out (bin),Pattern Name,Route Name,Transit Date,Transit Hour,Trip In Filter,Trip Out Filter,Trip Start Transit Hour,TripType,Vehicle,Vehicle Name Filter,ActualTime,Daily Trip ID,Start Trip Time,Number of Trips,Passengers In,Passengers Out,Trip Passengers In,Trip Passengers Out
0,2232,Heritage at Sixth Concession,Tuesday,EASTBOUND,September,1,0,PAT1 LASALLE 25 WEEKDAY 1 EAST,LASALLE 25,2024-09-03,12,True,True,11,Regular,8024,True,2024-09-03 12:10:34 PM,4474288,2024-09-03 11:54:25 AM,1,1,0,2,5
1,2168,Vollmer Centre Front Entrance,Tuesday,EASTBOUND,September,1,0,PAT1 LASALLE 25 WEEKDAY 1 EAST,LASALLE 25,2024-09-03,11,True,True,11,Regular,8024,True,2024-09-03 11:54:25 AM,4474288,2024-09-03 11:54:25 AM,1,1,0,2,5
2,2263,St Clair College Front Entrance,Tuesday,EASTBOUND,September,0,5,PAT1 LASALLE 25 WEEKDAY 1 EAST,LASALLE 25,2024-09-03,12,True,True,11,Regular,8024,True,2024-09-03 12:16:00 PM,4474288,2024-09-03 11:54:25 AM,1,0,5,2,5
3,2233,Sixth Concession at Montgomery,Tuesday,EASTBOUND,September,0,0,PAT1 LASALLE 25 WEEKDAY 1 EAST,LASALLE 25,2024-09-03,12,True,True,11,Regular,8024,True,2024-09-03 12:11:40 PM,4474288,2024-09-03 11:54:25 AM,1,0,0,2,5
4,2231,Heritage at Blackthorn,Tuesday,EASTBOUND,September,0,0,PAT1 LASALLE 25 WEEKDAY 1 EAST,LASALLE 25,2024-09-03,12,True,True,11,Regular,8024,True,2024-09-03 12:09:54 PM,4474288,2024-09-03 11:54:25 AM,1,0,0,2,5


In [10]:
dfs_dict['April 2024']['Route Name'].unique()

['TUNNEL',
 'CROSSTOWN 2',
 '755',
 '758',
 '751',
 'PARENT 14',
 'DOMINION 5',
 '518 EXPRESS',
 'ROUTE 605',
 '705',
 'LAUZON 10',
 'DOUGALL 6',
 '760',
 'TRANSWAY 1C',
 'WALKERVILLE 8',
 '709',
 '418 EXPRESS',
 '708',
 'LASALLE 25',
 '711',
 '706',
 'LTW ROUTE 42',
 '754',
 '707',
 '710',
 'CENTRAL 3',
 'TRANSWAY 1A',
 'OTTAWA 4',
 '702',
 'SOUTH WINDSOR 7',
 '756']

# Change in all the dfs DOMINION 5 for ROUTE 115

In [11]:
for key, df in dfs_dict.items():
    df['Route Name'] = df['Route Name'].apply(lambda x: 'ROUTE 115' if x == 'DOMINION 5' else x)

# Remove not useful columns

* Day of week, bins, transit hour, transit date, trip in, trip out, trip start transit hour, triptype, vehicle filter, number of trips, trip pass in, trip pass out

In [12]:
for key, _ in dfs_dict.items():
    # Use the `drop` method in Vaex to drop the columns
    if 'Day of Week' in dfs_dict[key].column_names:
        dfs_dict[key] = dfs_dict[key].drop(['Day of Week', 'Passengers In (bin)', 'Passengers Out (bin)', 'Transit Date', 'Transit Hour',
                                            'Trip In Filter', 'Trip Out Filter', 'Trip Start Transit Hour', 'TripType', 'Month',
                                            'Vehicle Name Filter', 'Number of Trips', 'Trip Passengers In', 'Trip Passengers Out',
                                            'Pattern Name'])

dfs_dict['March 2024'].head(5)

#,Stop Code,Stop Name,Direction,Route Name,Vehicle,ActualTime,Daily Trip ID,Start Trip Time,Passengers In,Passengers Out
0,2261,Devonshire Mall at Cineplex,NORTHBOUND,TRANSWAY 1A,8054,2024-03-22 4:31:25 PM,4220675,2024-03-22 4:31:25 PM,21,0
1,1046,Howard at Grand Marais,NORTHBOUND,TRANSWAY 1A,8054,2024-03-22 4:38:49 PM,4220675,2024-03-22 4:31:25 PM,3,0
2,1042,Howard at Edinborough,NORTHBOUND,TRANSWAY 1A,8054,2024-03-22 4:39:54 PM,4220675,2024-03-22 4:31:25 PM,2,1
3,1039,Howard at Hildegard,NORTHBOUND,TRANSWAY 1A,8054,2024-03-22 4:41:21 PM,4220675,2024-03-22 4:31:25 PM,2,0
4,1041,Howard at McDougall,NORTHBOUND,TRANSWAY 1A,8054,2024-03-22 4:40:36 PM,4220675,2024-03-22 4:31:25 PM,1,0


# Calculate Passenger Transfer

In [13]:
for key, df in dfs_dict.items():
    df['Passengers Transfer'] = df['Passengers In'] - df['Passengers Out']

# Add Actual Bus Occupancy column

In [14]:
def net_passenger(data):
    # Convert Vaex DataFrame to Pandas DataFrame
    data = data.to_pandas_df()

    # Sort the DataFrame by 'Route Name', 'Daily Trip ID', and 'ActualTime'
    data = data.sort_values(by=['Route Name', 'Daily Trip ID', 'ActualTime'])

    # Calculate 'Passengers Transfer'
    data['Passengers Transfer'] = data['Passengers In'] - data['Passengers Out']

    # Calculate cumulative sum of 'Passengers Transfer' for each 'Route Name' and 'Daily Trip ID'
    data['Actual Bus Occupancy'] = data.groupby(['Route Name', 'Daily Trip ID'])['Passengers Transfer'].cumsum()

    # Group the data once to avoid repeated filtering inside the loop
    grouped = data.groupby('Daily Trip ID', group_keys=False)

    # Process each group
    def process_trip(trip_data):
        # Get final occupancy
        final_occupancy = trip_data['Actual Bus Occupancy'].iloc[-1]
        passengers_transfer = trip_data['Passengers Transfer'].values
        occupancy_cumsum = trip_data['Actual Bus Occupancy'].values

        # Adjust based on final occupancy
        if final_occupancy < 0:
            # Find minimum negative value and adjust the first occurrence
            min_value = occupancy_cumsum.min()
            first_negative_idx = occupancy_cumsum.argmin()  # Find position in the array
            passengers_transfer[first_negative_idx] += abs(min_value)

        elif final_occupancy > 0:
            # Find the first occurrence of maximum occupancy and adjust
            max_value = occupancy_cumsum.max()
            first_max_idx = occupancy_cumsum.argmax()  # Find position in the array
            passengers_transfer[first_max_idx] -= final_occupancy

        # Update adjusted cumulative sum
        adjusted_cumsum = passengers_transfer.cumsum()

        # Ensure no negative occupancy values remain
        while (adjusted_cumsum < 0).any():
            min_value = adjusted_cumsum.min()
            first_negative_idx = (adjusted_cumsum < 0).argmax()  # Find position in the array
            passengers_transfer[first_negative_idx] += abs(min_value)
            adjusted_cumsum = passengers_transfer.cumsum()

        # Update the DataFrame with the adjusted values
        trip_data['Passengers Transfer'] = passengers_transfer
        trip_data['Adjusted Bus Occupancy'] = adjusted_cumsum

        return trip_data

    # Apply the processing function to each group and collect the results
    data = grouped.apply(process_trip, include_groups=False)

    # Replace Actual Bus Occupancy with the adjusted and remove the temporary column
    data['Actual Bus Occupancy'] = data['Adjusted Bus Occupancy']
    data = data.drop(columns=['Adjusted Bus Occupancy'])

    # Convert back to Vaex DataFrame
    data_vaex = vaex.from_pandas(data)

    return data_vaex


In [15]:
for key, df in dfs_dict.items():
    print(f'Processing: {key} data')
    dfs_dict[key] = net_passenger(df)

Processing: October 2023 data
Processing: Jan 2024 data
Processing: October 2024 data
Processing: July 2024 data
Processing: June 2024 data
Processing: September 2024 data
Processing: August 2024 data
Processing: December 2023 data
Processing: Feb 2024 data
Processing: November 2023 data
Processing: March 2024 data
Processing: April 2024 data
Processing: May 2024 data


# Create a daily trip sequence, bus order during the day

In [16]:
def assign_daily_order_trip_id_by_route_time_direction_vaex(data):
    # Convert Vaex DataFrame to Pandas
    df = data.to_pandas_df()

    # Parse datetime explicitly with AM/PM handling
    df['Start Trip Time'] = pd.to_datetime(
        df['Start Trip Time'], 
        format='%Y-%m-%d %I:%M:%S %p',  # Explicitly account for AM/PM
        errors='coerce'  # Handle invalid timestamps gracefully
    )

    # Subtract 4 hours to align trips with the same "Business Day"
    df['Adjusted Start Time'] = df['Start Trip Time'] - timedelta(hours=4)

    # Extract the adjusted date for grouping
    df['Adjusted Date'] = df['Adjusted Start Time'].dt.date  # Keeps datetime.date format

    # Sort by Route Name, Direction, Adjusted Date, and Start Trip Time
    df = df.sort_values(by=['Route Name', 'Direction', 'Adjusted Date', 'Start Trip Time']).reset_index(drop=True)

    # Assign daily_order_trip_id by grouping on Route Name, Direction, Adjusted Date, and Start Trip Time
    df['daily_order_trip_id'] = (
        df.groupby(['Adjusted Date', 'Route Name', 'Direction'])['Start Trip Time']
        .transform('rank', method='dense').astype(int)
    )

    # Rename Adjusted Date to Business Day without changing its format
    df = df.rename({'Adjusted Date': 'Business Day'}, axis=1)

    # Drop temporary columns if not needed
    df = df.drop(columns=['Adjusted Start Time'])
    
    # Convert Business Day to pandas datetime before formatting
    df['Business Day'] = pd.to_datetime(df['Business Day'])  # Convert datetime.date to pandas datetime
    df['Business Day'] = df['Business Day'].dt.strftime('%Y-%m-%d')  # Format as string in 'YYYY-MM-DD'

    # Convert back to Vaex DataFrame
    df = vaex.from_pandas(df)
    df['Start Trip Time'] = df['Start Trip Time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    return df



In [17]:
for key, df in dfs_dict.items():
    print(f'Processing: {key} data')
    dfs_dict[key] = assign_daily_order_trip_id_by_route_time_direction_vaex(df)

Processing: October 2023 data
Processing: Jan 2024 data
Processing: October 2024 data
Processing: July 2024 data
Processing: June 2024 data
Processing: September 2024 data
Processing: August 2024 data
Processing: December 2023 data
Processing: Feb 2024 data
Processing: November 2023 data
Processing: March 2024 data
Processing: April 2024 data
Processing: May 2024 data


In [18]:
#dfs_dict['March 2024'].head(5)

# Convert Actual Time column to be in the 24H format

In [19]:
def convert_to_24h(df):
    # Convert Vaex DataFrame to Pandas DataFrame
    pandas_df = df.to_pandas_df()
    
    # Ensure 'ActualTime' is datetime and convert to 24-hour format
    pandas_df['ActualTime'] = pd.to_datetime(pandas_df['ActualTime'], format='%Y-%m-%d %I:%M:%S %p', errors='coerce')
    pandas_df['ActualTime'] = pandas_df['ActualTime'].dt.strftime('%Y-%m-%d %H:%M:%S')
    
    # Convert back to Vaex DataFrame
    vaex_df = vaex.from_pandas(pandas_df)
    return vaex_df


In [20]:
for key, df in dfs_dict.items():
    print(f'Processing: {key} data')
    dfs_dict[key] = convert_to_24h(df)

Processing: October 2023 data
Processing: Jan 2024 data
Processing: October 2024 data
Processing: July 2024 data
Processing: June 2024 data
Processing: September 2024 data
Processing: August 2024 data
Processing: December 2023 data
Processing: Feb 2024 data
Processing: November 2023 data
Processing: March 2024 data
Processing: April 2024 data
Processing: May 2024 data


In [21]:
#dfs_dict['December 2023'].head(5)

# Remove rows that are not in the direction map

In [22]:
direction_map = {'TRANSWAY 1A': {0: 'NORTHBOUND', 1: 'SOUTHBOUND'},
 'TRANSWAY 1C': {1: 'EASTBOUND', 0: 'WESTBOUND'},
 'CROSSTOWN 2': {0: 'WESTBOUND', 1: 'EASTBOUND'},
 'CENTRAL 3': {1: 'EASTBOUND', 0: 'WESTBOUND'},
 'OTTAWA 4': {0: 'WESTBOUND', 1: 'EASTBOUND'},
 'DOUGALL 6': {1: 'SOUTHBOUND', 0: 'NORTHBOUND'},
 'WALKERVILLE 8': {1: 'SOUTHBOUND', 0: 'NORTHBOUND'},
 'PARENT 14': {0: 'NORTHBOUND', 1: 'SOUTHBOUND'},
 'TUNNEL': {1: 'SOUTHBOUND', 0: 'NORTHBOUND'},
 'SOUTH WINDSOR 7': {0: 'WESTBOUND', 1: 'EASTBOUND'},
 'LAUZON 10': {1: 'SOUTHBOUND', 0: 'NORTHBOUND'},
 'LASALLE 25': {0: 'WESTBOUND', 1: 'EASTBOUND'},
 'LTW ROUTE 42': {1: 'SOUTHBOUND', 0: 'NORTHBOUND'},
 '518 EXPRESS': {0: 'WESTBOUND', 1: 'EASTBOUND'},
 'ROUTE 605': {0: 'NORTHBOUND', 1: 'SOUTHBOUND'},
 '418 EXPRESS': {0: 'WESTBOUND', 1: 'EASTBOUND'},
 'ROUTE 115': {0: 'NORTHBOUND', 1: 'SOUTHBOUND'}}

In [23]:
routes = [route for route in direction_map.keys()]

In [24]:
for key, df in dfs_dict.items():
    dfs_dict[key] = df[df['Route Name'].isin(routes)]

# Concatenate the latitude and longitud columns to the datasets

In [25]:
expected.head(5)

,route_long_name,stop_name,stop_code,stop_lat,stop_lon,arrival_time,departure_time,stop_sequence,trip_id,direction_id
0,TRANSWAY 1A,Windsor Transit Terminal,1000,42.317486,-83.043368,17:50:00,17:50:00,1,1133706,1
1,TRANSWAY 1A,Ouellette at Chatham,1002,42.317882,-83.039374,17:52:00,17:52:00,2,1133706,1
2,TRANSWAY 1A,Ouellette at Park,1004,42.315935,-83.038091,17:53:00,17:53:00,3,1133706,1
3,TRANSWAY 1A,Ouellette at Wyandotte,1006,42.313499,-83.036628,17:54:00,17:54:00,4,1133706,1
4,TRANSWAY 1A,Ouellette at Elliott,1008,42.311320,-83.035171,17:59:00,17:59:00,5,1133706,1


In [26]:
df = dfs_dict['March 2024'].copy()
df.head(5)

#,Stop Code,Stop Name,Direction,Route Name,Vehicle,ActualTime,Start Trip Time,Passengers In,Passengers Out,Passengers Transfer,Actual Bus Occupancy,Business Day,daily_order_trip_id
0,1834,Hotel Dieu Grace Healthcare,EASTBOUND,418 EXPRESS,8069,2024-03-01 06:41:52,2024-03-01 06:41:52,1,0,1,1,2024-03-01,1
1,2090,Prince at College,EASTBOUND,418 EXPRESS,8069,2024-03-01 06:46:41,2024-03-01 06:41:52,0,0,0,1,2024-03-01,1
2,1215,Sandwich at South,EASTBOUND,418 EXPRESS,8069,2024-03-01 06:48:27,2024-03-01 06:41:52,1,0,1,2,2024-03-01,1
3,1219,Sandwich at Brock,EASTBOUND,418 EXPRESS,8069,2024-03-01 06:49:29,2024-03-01 06:41:52,1,0,1,3,2024-03-01,1
4,1222,Wyandotte at Mill,EASTBOUND,418 EXPRESS,8069,2024-03-01 06:51:03,2024-03-01 06:41:52,4,0,4,7,2024-03-01,1


In [27]:
expected = expected[expected['route_long_name'].isin(routes)]
unique_stop_codes = expected.drop_duplicates(subset=['stop_code'], keep='first')[['stop_code', 'stop_lat', 'stop_lon']]
unique_stop_codes = vaex.from_pandas(unique_stop_codes)
unique_stop_codes.head(5)

#,stop_code,stop_lat,stop_lon
0,1000,42.3175,-83.0434
1,1002,42.3179,-83.0394
2,1004,42.3159,-83.0381
3,1006,42.3135,-83.0366
4,1008,42.3113,-83.0352


In [28]:
# Perform the join operation
for key, df in dfs_dict.items():
    dfs_dict[key] = df.join(
        unique_stop_codes,          # The right DataFrame
        left_on='Stop Code',        # Column in the left DataFrame (df)
        right_on='stop_code',       # Column in the right DataFrame (unique_stop_codes)
        how='left'                  # Perform a left join
        )

dfs_dict['June 2024'].head(5)

#,Stop Code,Stop Name,Direction,Route Name,Vehicle,ActualTime,Start Trip Time,Passengers In,Passengers Out,Passengers Transfer,Actual Bus Occupancy,Business Day,daily_order_trip_id,stop_code,stop_lat,stop_lon
0,1834,Hotel Dieu Grace Healthcare,EASTBOUND,418 EXPRESS,8070,2024-06-03 06:42:25,2024-06-03 06:42:25,0,0,0,0,2024-06-03,1,1834,42.286447,-83.064606
1,2090,Prince at College,EASTBOUND,418 EXPRESS,8070,2024-06-03 06:46:37,2024-06-03 06:42:25,0,0,0,0,2024-06-03,1,2090,42.289898,-83.075598
2,9622,Peter at Chippawa,EASTBOUND,418 EXPRESS,8070,2024-06-03 06:48:30,2024-06-03 06:42:25,1,0,1,1,2024-06-03,1,--,--,--
3,9623,Peter at Brock,EASTBOUND,418 EXPRESS,8070,2024-06-03 06:49:09,2024-06-03 06:42:25,0,0,0,1,2024-06-03,1,--,--,--
4,1222,Wyandotte at Mill,EASTBOUND,418 EXPRESS,8070,2024-06-03 06:50:01,2024-06-03 06:42:25,1,0,1,2,2024-06-03,1,1222,42.30079002,-83.07097849


# Make the vaex file and export them as csv and also add a month column

In [36]:
# Iterate over each DataFrame in the dictionary
for key, df in dfs_dict.items():
    # Copy the DataFrame to avoid modifying the original
    df_copy = df.copy()
    
    # Determine the full month name based on the dictionary key
    month_name = key.split()[0]  # Extract the month part from the key
    if month_name == "Jan":
        month_name = "January"
    elif month_name == "Feb":
        month_name = "February"

    # Generate a constant array of the month name matching the unfiltered length
    full_array = np.full(df_copy.length_unfiltered(), month_name, dtype='U10')

    # Add a physical column to the DataFrame
    df_copy.add_column('month', full_array)

    # Update the dictionary with the modified DataFrame
    dfs_dict[key] = df_copy

In [37]:
# Define the folder name
folder_name = "cleaned_datasets"

# Create the folder if it doesn't exist
os.makedirs(folder_name, exist_ok=True)

# Iterate over the dictionary and save each DataFrame as a CSV
for key, df in dfs_dict.items():
    # Replace spaces in the key with underscores
    file_name = key.replace(" ", "_") + ".csv"
    
    # Define the full path for the CSV
    file_path = os.path.join(folder_name, file_name)
    
    # Save the DataFrame as a CSV
    df.export_csv(file_path)
    
    print(f"Saved {file_path}")

Saved cleaned_datasets/October_2023.csv
Saved cleaned_datasets/Jan_2024.csv
Saved cleaned_datasets/October_2024.csv
Saved cleaned_datasets/July_2024.csv
Saved cleaned_datasets/June_2024.csv
Saved cleaned_datasets/September_2024.csv
Saved cleaned_datasets/August_2024.csv
Saved cleaned_datasets/December_2023.csv
Saved cleaned_datasets/Feb_2024.csv
Saved cleaned_datasets/November_2023.csv
Saved cleaned_datasets/March_2024.csv
Saved cleaned_datasets/April_2024.csv
Saved cleaned_datasets/May_2024.csv


# Add the Expected Arrive Time column to each month